In [1]:
from PIL import Image
#Getting all the files from radarc
import os
import pandas as pd
directory1 = 'Radarc'
directory2 = 'samples\\CAM_FRONT'
# list all files in the directory
radar_filenames = os.listdir(directory1)
image_filenames = os.listdir(directory2)
# print the list of files
image_filenames

['n008-2018-08-22-15-53-49-0400__CAM_FRONT__1534967832362404.jpg',
 'n008-2018-08-22-15-53-49-0400__CAM_FRONT__1534967832862404.jpg',
 'n008-2018-08-22-15-53-49-0400__CAM_FRONT__1534967833362404.jpg',
 'n008-2018-08-22-15-53-49-0400__CAM_FRONT__1534967833862404.jpg',
 'n008-2018-08-22-15-53-49-0400__CAM_FRONT__1534967834362404.jpg',
 'n008-2018-08-22-15-53-49-0400__CAM_FRONT__1534967834862404.jpg',
 'n008-2018-08-22-15-53-49-0400__CAM_FRONT__1534967835362404.jpg',
 'n008-2018-08-22-15-53-49-0400__CAM_FRONT__1534967835862404.jpg',
 'n008-2018-08-22-15-53-49-0400__CAM_FRONT__1534967836362404.jpg',
 'n008-2018-08-22-15-53-49-0400__CAM_FRONT__1534967836862404.jpg',
 'n008-2018-08-22-15-53-49-0400__CAM_FRONT__1534967837362404.jpg',
 'n008-2018-08-22-15-53-49-0400__CAM_FRONT__1534967837912423.jpg',
 'n008-2018-08-22-15-53-49-0400__CAM_FRONT__1534967838412404.jpg',
 'n008-2018-08-22-15-53-49-0400__CAM_FRONT__1534967838912404.jpg',
 'n008-2018-08-22-15-53-49-0400__CAM_FRONT__1534967839412404.j

In [2]:
print(len(radar_filenames))
print(len(image_filenames))

3409
3409


In [3]:
import numpy as np
from scipy import ndimage
from PIL import Image
import cv2
import matplotlib.pyplot as plt
channel1=[]
channel2=[]
channel3=[]
#channel4=[]
#range(len(image_filenames))
for k in range(len(image_filenames)):
    # Load the grayscale images of camera and radar image
    #image = Image.open('samples\\CAM_FRONT\\'+image_filenames[k]).convert('L')
    image = cv2.imread('samples\\CAM_FRONT\\'+image_filenames[k], cv2.IMREAD_GRAYSCALE)
    image_array = np.array(image)
    channel1.append(image_array)
    #image = Image.open('Radarc\\'+image_filenames[k]).convert('L')
    image = cv2.imread('Radarc\\'+image_filenames[k], cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (1600, 900))
    image_array = np.array(image)
    #channel4.append(image_array)
    #channel2.append(image_array)
    # Define the Gaussian kernel for smoothing radar image
    sigma = 6
    kernel_size = 20
    kernel = np.zeros((kernel_size, kernel_size))
    for i in range(kernel_size):
        for j in range(kernel_size):
            kernel[i,j] = np.exp(-((i-kernel_size//2)**2 + (j-kernel_size//2)**2)/(2*sigma**2))
    # Normalize the kernel
    kernel /= np.sum(kernel)
    # Apply the Gaussian filter to the image
    filtered_image = ndimage.convolve(image_array, kernel)
    channel2.append(filtered_image.astype(np.uint8))
    # Save the filtered image
    filtered_image = Image.fromarray(filtered_image.astype(np.uint8))
    #Making Channel3
    # Load grayscale image
    img_gray = cv2.imread('Radarc\\'+image_filenames[k], cv2.IMREAD_GRAYSCALE)
    img_gray = cv2.resize(img_gray, (1600, 900))
    # Apply binary thresholding
    _, img_binary = cv2.threshold(img_gray, 120, 255, cv2.THRESH_BINARY)
    # Apply distance transform
    img_distance = cv2.distanceTransform(img_binary, cv2.DIST_L2, 3)
    # Scale distance transform values to [0, 255] range for visualization
    img_distance = cv2.normalize(img_distance, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    channel3.append(img_distance)
    # Save the output image
    #cv2.imwrite('Distance_trans\\'+image_filenames[k], img_distance)
    # Merge the 3 images into a single 3-channel image
    channel2[k]=cv2.multiply(channel2[k], 0.6)
    channel3[k]=cv2.multiply(channel3[k], 0.4)
    merged_img = cv2.merge((channel1[k], channel2[k], channel3[k]))
    # Save the new image
    cv2.imwrite('GRD\\'+image_filenames[k], merged_img)
    #print(k)
    #filtered_image.save('radar_mask3.jpg')

In [4]:
print(len(channel1[0][0]))
#print(len(channel4[0][0]))
print(len(channel2[3][0]))
print(len(channel3[0][0]))

1600
1600
1600


In [5]:
# from PIL import Image

# # Load the image
# img = Image.open('Radarc\\'+image_filenames[0])

# # Get the size of the image
# width, height = img.size

# # Print the size of the image
# print(f'The size of the image is: {width} x {height}')
